Setup code

In [266]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY in .env file
# UPSTAGE_API_KEY=your_api_key

import warnings
warnings.filterwarnings("ignore")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [267]:
text = """
The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 Englishto-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.
"""

query = f"""
Make a 3 problem in this abstract.

{text}
"""

query2 = """
Linear transformation is a function that satisfies two features.
One is homogeneity, and other is additivity.

Linear transformation of the subspace is also subspace.
Prove or disprove this statement based on given contents.
"""

query3 = """
The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""

query4 = """
What is a Von Neumman structure?
"""

In [268]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatUpstage()

message = query2

prompt_template = PromptTemplate.from_template(
    """
You are the AI that separates instruction and context from the query.
In this query, separate the instruction and context. Instruction and context must not have duplicate contents.
Only use given query itself, do not generate or change any words from the query.
Instruction is saying about what AI should do.
Context is can be a base information or some text to convert or manipulate.
Context can be not provided, then print context to not provided. Do not make context.
Print instruction as first paragraph and context as second paragraph.
---
Query : {query}
"""
)
chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"query": message})

instr, context = [s.strip() for s in answer.split("Instruction:")[1].split("Context:")]
print(instr)
print(context)

Prove or disprove the statement "Linear transformation of the subspace is also subspace" based on the given contents.
Linear transformation is a function that satisfies two features. One is homogeneity, and other is additivity.


In [269]:
import os
from tavily import TavilyClient

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

scontext = str(tavily.search(query=instr))

prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", divide instruction into few steps.
Do not answer or calculate for each steps, and only divide instructions to the smaller problem.
Only print 1., 2., 3., ... steps to do. Each step should be a one-liner.
Refer to the context if you need : {context}
Searched context to help dividing instructions : {scontext}
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context, "scontext": scontext}).split("\n")
print(answer)
print(len(answer))

['1. Recall the definition of a linear transformation.', '2. Define what a subspace is.', '3. Apply the definition of a linear transformation to a subspace.', '4. Determine if the result of the linear transformation is still a subspace.', '5. Provide a proof or counterexample to support your conclusion.']
5


In [270]:
prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", determine instruction can be done only using context.
Just answer it in "Yes" or "No".
---
Context : {context}
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context})
print(answer)

add = ""

if(answer.startswith("No")): # find context from tavily
    context = scontext
    add = "\nAnswer with references if it exists."

No


In [271]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

rag_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
    ]
)

chain = rag_with_history_prompt | llm | StrOutputParser()

history = []

for i in range(len(answer)):
    if (i == 0 and len(context) != 0):
        history.append(
            HumanMessage(answer[i] + "\n Refer to or use this context.\n Context:" + context)
        )
    else:
        history.append(
            HumanMessage(answer[i]) 
        )
    chain_result = chain.invoke({"history": history})
    history.append(
        AIMessage(chain_result)
    )

history.append(
    HumanMessage(message + add + "\nIf answer is not context-based, then only print 'I don't know about that.'")
)
answer = chain_result = chain.invoke({"history": history})

print(answer)


The statement "Linear transformation of the subspace is also subspace" is true. This can be proven based on the definition of a linear transformation and a subspace.

A linear transformation is a function that satisfies two properties: homogeneity and additivity. Homogeneity means that for any scalar c and vector v in the domain of the transformation, the transformation maps cv to c times the transformation of v. Additivity means that for any two vectors v and w in the domain of the transformation, the transformation maps v + w to the sum of the transformations of v and w.

Now, let's consider a linear transformation T applied to a subspace S of a vector space V. Since T is a linear transformation, it preserves both addition and scalar multiplication. This means that for any vectors v and w in S and any scalar c, we have:

1. T(v + w) = T(v) + T(w)
2. T(cv) = cT(v)

Since S is a subspace, it is closed under addition and scalar multiplication. Therefore, T(v + w) and T(cv) will also be 